## 손글씨 이미지 데이터 MNIST
- 많은 이미지 데이터 학습
- 학습 후 새로운 데이터 입력 시 판별
- 지도 학습
- 데이터는 학습용 6만개, 테스트 전용 1만개
- http://yann.lecun.com/exdb/mnist/  
train-images-idx3-ubyte.gz:  training set images (9912422 bytes)  
train-labels-idx1-ubyte.gz:  training set labels (28881 bytes)   
t10k-images-idx3-ubyte.gz:   test set images (1648877 bytes)   
t10k-labels-idx1-ubyte.gz:   test set labels (4542 bytes)

### 1. 연구목표

### 2. 데이터 획득/수집
- 손글씨 이미지(숫자 0~9)를 학습시켜서, 이후에 새로운 손글씨 이미지를 판별해내는 머신러닝 모듈 구현
- 이미지 데이터를 디코딩하고, 이미지 하나당 구성된 총 픽셀 수 및 사이즈를 계산하여 이미지를 추출 → 디코딩 → 6만개의 데이터 생성
- 이미지와 같은 바이너리 데이터를 벡터화하는 과정

### 3. 데이터 준비/통찰/전처리
- http://yann.lecun.com/exdb/mnist/ 접속
- (데이터 수집 level3) 웹 스크래핑을 활용하여 압축 데이터를 받는 URL 획득
- 압축 데이터를 직접 요청하여 받고 저장하면서 압축 풀기 등 일련의 과정 진행

### 4. 데이터 탐색/통찰/시각화
- MNIST database 구조 이해
- 디코딩 처리 가능
- 엔디언 내용 추가 이해

### 5. 데이터 모델링 or 모델 구축

### 6. 시스템 통합 or 솔루션/서비스/레포트

#### 2. 데이터 획득/수집
- 2-1. 수집할 데이터 url 획득

In [1]:
import urllib.request as req
from bs4 import BeautifulSoup

In [2]:
target_site = 'http://yann.lecun.com/exdb/mnist/'
target_site

'http://yann.lecun.com/exdb/mnist/'

In [3]:
page = req.urlopen(target_site)
page

In [4]:
# soup 생성
soup = BeautifulSoup (page, 'html5lib')
soup

<html><head>
   <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
   <title>MNIST handwritten digit database, Yann LeCun, Corinna Cortes and Chris Burges</title>
<script type="text/javascript">
var gaJsHost = (("https:" == document.location.protocol) ? "https://ssl." : "http://www.");
document.write(unescape("%3Cscript src='" + gaJsHost + "google-analytics.com/ga.js' type='text/javascript'%3E%3C/script%3E"));
</script><script type="text/javascript">
var pageTracker = _gat._getTracker("UA-6178702-1");
pageTracker._trackPageview();
</script></head>




<body alink="#111111" background="wm5b.gif" bgcolor="#FFFFFF" link="#FF0000" vlink="#820594">

<center>
<h1>
<font color="#FF0000"><font size="+4">THE MNIST DATABASE</font></font></h1></center>

<center><font color="#FF0000"><font size="+4">of handwritten digits</font></font></center>

<center><font color="#FF0000"><font size="+4"></font></font></center>

<center><font color="#3366FF"><font size="+1"><a href="http://yann

In [11]:
# gz 파일 4개 URL 획득
# for tt in soup.find_all('tt') :
#     if tt.a :
#         print(tt.a['href'])
#         # print(tt.a.string)

for tt in soup.find_all('tt')[:4] :
    print(tt.a['href'])
    # print(tt.a.string)

train-images-idx3-ubyte.gz
train-labels-idx1-ubyte.gz
t10k-images-idx3-ubyte.gz
t10k-labels-idx1-ubyte.gz


In [12]:
# 리스트 내포
files = [ tt.a['href'] for tt in soup.find_all('tt')[:4] ]
files

['train-images-idx3-ubyte.gz',
 'train-labels-idx1-ubyte.gz',
 't10k-images-idx3-ubyte.gz',
 't10k-labels-idx1-ubyte.gz']

- 압축 데이터 다운로드

In [13]:
import os, os.path, gzip

In [14]:
# 파일 저장 위치 선정
savePath = './data/mnist'
# 해당 디렉토리가 없으면 생성
if not os.path.exists(savePath) :
    os.makedirs(savePath)

In [16]:
# 저장
# 진행바 표시
from tqdm import tqdm_notebook
for file in tqdm_notebook(files) :
    print('원본 데이터 주소', target_site + file )
    local_path = '%s/%s' % ( savePath, file )
    print('저장될 데이터의 경로', local_path )
    # 이미 파일이 존재하면 새로 받지 않겠다
    if not os.path.exists( local_path ) :
        # 웹상에 존재하는 리소스를 로컬에 직접 저장(위치 지정)
        req.urlretrieve( target_site + file, local_path )

원본 데이터 주소 http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
저장될 데이터의 경로 ./data/mnist/train-images-idx3-ubyte.gz
원본 데이터 주소 http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
저장될 데이터의 경로 ./data/mnist/train-labels-idx1-ubyte.gz
원본 데이터 주소 http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
저장될 데이터의 경로 ./data/mnist/t10k-images-idx3-ubyte.gz
원본 데이터 주소 http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz
저장될 데이터의 경로 ./data/mnist/t10k-labels-idx1-ubyte.gz



#### 2-2. 압축 데이터 압축 해제

In [18]:
# 압축 해제 시 이름 : train-images-idx3-ubyte.gz -> train-images-idx3-ubyte
for file in tqdm_notebook(files) :
    # 원본 파일
    ori_file = '%s/%s' % (savePath, file)
    # 압축 해제 파일
    raw_file = '%s/%s' % (savePath, file[:-3])
    print(ori_file, raw_file)
    # 압축 파일을 오픈 → 읽고 → 기록
    # 압축 파일 오픈
    with gzip.open(ori_file, 'rb') as fg :
        # 내용을 읽고 (대용량이면 분할해서 읽어야 한다)
        tmp = fg.read()
        with open(raw_file, 'wb') as f :
            f.write(tmp)

./data/mnist/train-images-idx3-ubyte.gz ./data/mnist/train-images-idx3-ubyte
./data/mnist/train-labels-idx1-ubyte.gz ./data/mnist/train-labels-idx1-ubyte
./data/mnist/t10k-images-idx3-ubyte.gz ./data/mnist/t10k-images-idx3-ubyte
./data/mnist/t10k-labels-idx1-ubyte.gz ./data/mnist/t10k-labels-idx1-ubyte



#### 3. 데이터 준비/통찰/전처리
- DB 구조 이해상 나온 내용
- 엔디언(endian) : 컴퓨터 메모리와 같은 1차원의 공간에 여러 개의 연속된 데이터를 배열하는 방법 → ordering(오더링) → byte order(바이트 오더 : 바이트 배열 방법)
- 값 : 0x12345678

#### 빅 엔디언(Big-endian)
- 큰 단위가 앞에 나온다
- 최상위 바이트(MSB - Most Significant Byte)부터 차례로 저장하는 방식

|메모리|...|0x100|0x101|0x102|0x103|...|
|-|-|-|-|-|-|-|
|값|...|0x12|0x34|0x56|0x78|...|

#### 리틀 엔디언(Little-endian)
- 작은 단위가 앞에 나온다
- 최하위 바이트(LSB - Least Significant Byte)부터 차례로 저장하는 방식

|메모리|...|0x100|0x101|0x102|0x103|...|
|-|-|-|-|-|-|-|
|값|...|0x78|0x56|0x34|0x12|...|

#### label의 구조
- 구조
 > magic number : 4 byte  
 > label 개수 : 4 byte  
 > label : 1 byte, ...  

#### images 구조
- 구조
 > magic number : 4 byte  
 > image 개수 : 4 byte  
 > 이미지 가로 : 4 byte  
 > 이미지 세로 : 4 byte  
 > 이미지 : 28 * 28 byte, ...

In [ ]:
# 레이블데이터, 원데이터를 읽고
# 풀어서 데이터를 저장

# train-images-idx3-ubyte
# train-labels-idx1-ubyte
# t10k-images-idx3-ubyte
# t10k-labels-idx1-ubyte

# 훈련용 데이터, 테스트용 데이터 각각 처리 → 같은 구조로 2세트 → 함수로 처리
# 실데이터는 어디에 저장할 것인가? → csv로 저장(*.pgm)

In [19]:
import struct

In [40]:
def decode_mnist_rawData( dataType = 'train', maxCnt = 0 ) :
    # 레이블 파일 읽기 모드 오픈
    label_f = open('%s/%s-labels-idx1-ubyte' % (savePath, dataType), 'rb')
    # 이미지 데이터 파일 읽기 모드 오픈
    image_f = open('%s/%s-images-idx3-ubyte' % (savePath, dataType), 'rb')
    # csv 쓰기 모드 오픈(train.csv)
    csv_f = open('%s/%s.csv' %(savePath, dataType), 'w', encoding = 'utf-8')
    # ------------------------------------------------------------------------
    
    # 디코딩을 위한 헤더 정보 추출
    # 매직 코드나 사이즈, 이미지의 가로, 세로 정보 등을 획득하기 위해서는
    # 4바이트씩 읽어서 빅 엔디언(Big endian) 처리를 해야한다
    # → struct 모듈이 해결
    
    # label
    # >II : 읽은 총 데이터를 2번 나눠서 각각 변수에 담는다
    magic, label_count = struct.unpack( '>II', label_f.read(8) )
    print(magic, label_count)
    
    # image : 파일 포인터(커서) 위치는 16
    magic, image_count, row, col = struct.unpack( '>IIII', image_f.read(16) )
    print(magic, image_count, row, col)
    # 이미지 한 개당 총 픽셀 수
    pixels = row * col
    
    # 데이터를 추출(디코딩)해서 csv에 저장
    for idx in range( label_count ) :
        # 데이터를 원하는 만큼 세팅했다면
        if idx >= maxCnt :
            break
        
        # 정답 추출 → 1 byte만 읽으면 된다 → unsigned byte → 'B'
        label_tmp = struct.unpack( 'B', label_f.read(1) )
        # print(label_tmp)
        # 값이 결측치에 해당될 경우
        # → 이미지 데이터도 같이 날린다
        if len(label_tmp) != 1 :
            continue
        # 라벨값, 정답 획득
        label = label_tmp[0]
        # -------------------------------------------------------------------
        
        # 이미지 추출 → 얼만큼 읽어야 하는가?
        # 이미지 데이터 원본은 엔디언과는 관계없다
        # 여러 바이트를 쓰는 정수가 아닌 데이터 자체이므로 무관
        binaryData = image_f.read(pixels)
        # print(type(binaryData))
        # 픽셀값 하나하나를 문자열로 만들어서 리스트에 담는다
        # strData = list(map(함수, 데이터))
        strData = list(map(lambda x : str(x), binaryData))
        # print(strData)
        # csv에 기록 : label, 이미지 데이터 줄바꿈
        csv_f.write( str(label)+'.' )
        csv_f.write( '.'.join(strData)+'\n' )
        
        # 해당 데이터를 별도로 저장하여 이미지 파일로 보관
        # 대략 10개 정도만 저장
        if idx < 10 :
            # pgm 파일로 저장
            fName = '{0}/{1}-{2}-{3}.pgm'.format( savePath, dataType, idx, label )
            # pgm 헤더
            header = 'P2 28 28 255\n'
            s = header + ' '.join( strData )
            # 파일 기록
            with open(fName, 'w', encoding = 'utf-8') as f :
                f.write(s)
    
    # 파일 닫기
    label_f.close()
    image_f.close()
    csv_f.close()

In [41]:
decode_mnist_rawData( dataType='train', maxCnt = 750 )
decode_mnist_rawData( dataType='t10k',  maxCnt = 250 )

2049 60000
2051 60000 28 28
2049 10000
2051 10000 28 28


#### 데이터 탐색/통찰/시각화

- 데이터는 현재 csv에 저장되어 있다. 이를 로드해서 데이터를 전처리하여 학습에 적합한 구조로 정리

In [42]:
# 저장된 csv 파일 경로
savePath + '/' + 'train' + '.csv', savePath + '/' + 't10k' + '.csv'

('./data/mnist/train.csv', './data/mnist/t10k.csv')

In [ ]:
# 1. csv를 오픈
# 2. csv를 읽는다 → 데이터 한 줄(row)을 읽어서 리스트에 멤버로 담는다
#    ex) train.csv를 리스트의 750개의 멤버가 생성, t10k는 250개 생성
#    [ [], [], [], ...  ]
# 3. 리스트의 멤버(리스트)는 0번 멤버가 레이블, 1번 ~ 마지막까지가 픽셀
#    픽셀은 현재 0 ~ 255까지의 값으로 구성
#    학습 성능을 높이기 위해 픽셀값을 0 ~ 1(255)로 정규화하여 배치, 1 + 28 * 28개
# { 'labels' : labels, 'images' : images }

In [48]:
def load_csv( dataType = 'train' ) :
    # 1. csv를 오픈
    fName = '{}/{}.csv'.format(savePath, dataType)
    print(fName)
    labels = []
    images = []
    with open(fName, 'r') as f :
        # 2. csv를 읽는다
        # print(f.readline)
        # for문을 이용하여 한 줄씩 read
        for line in f :
            # print(line)  # type : str
            tmp = line.split('.')
            print(tmp)
            # labels에 답(0~9) 데이터를 정수값으로 세팅
            labels.append(int(tmp[0]))
            # images에 [ 28*28개의 픽셀값 ] 멤버로 추가
            # 단, 데이터는 0~1까지 정규화된 값
            # 255로 정규화? 256으로 정규화
            # 데이터 자체를 들여다 보고
            # 데이터가 앞뒤로 영향을 받는 연속적인 성향을 가진 값인지, 독립적인 값인지 검토
            # 수치값 → 연속변수(255 : 최댓값 기준), 분류변수(256, 총 개수 기준)
            images.append( list(map(lambda x : int(x)/256, tmp[1:])) ) # tmp[1:]는 사본 → 메모리 낭비
    
    return { 'labels' : labels, 'images' : images }

In [50]:
# 최적화
def load_csv( dataType = 'train' ) :
    fName = '{}/{}.csv'.format( savePath, dataType )
    labels = []
    images = []
    with open(fName, 'r') as f :
        for line in f :
            tmp = line.split('.')
            # tmp.pop(0) : tmp의 0번 멤버를 제거하고 리턴
            labels.append(tmp.pop(0))
            images.append( list(map(lambda x : int(x)/256, tmp)) )
    return {'labels' : labels, 'images' : images } 

In [52]:
train = load_csv()
test = load_csv( dataType = 't10k' )

#### 5. 데이터 모델링 or 모델 구축

In [53]:
# 알고리즘 선택 → 파이프라인 구축
from sklearn import svm, model_selection, metrics

In [54]:
# 알고리즘 생성 → 파이프라인 구축, 하이퍼파라미터 튜닝
clf = svm.SVC()   # 기본값

In [ ]:
# 데이터 분류(이미 완료)

In [55]:
# 학습
clf.fit( train['images'], train['labels'] )

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [58]:
# 예측 (테스트 데이터 사용)
predict = clf.predict( test['images'] )
predict, len(predict)

(array(['7', '5', '1', '0', '4', '1', '4', '9', '2', '9', '0', '4', '9',
        '0', '1', '5', '4', '7', '5', '4', '9', '6', '4', '5', '4', '0',
        '7', '4', '0', '1', '3', '1', '3', '4', '7', '2', '7', '1', '1',
        '1', '1', '7', '4', '1', '5', '5', '1', '2', '4', '4', '6', '3',
        '5', '5', '2', '0', '4', '1', '9', '5', '7', '2', '9', '2', '7',
        '9', '2', '4', '3', '0', '7', '0', '2', '8', '1', '7', '3', '7',
        '9', '7', '9', '6', '2', '7', '8', '4', '7', '5', '6', '1', '3',
        '6', '9', '3', '1', '4', '1', '9', '6', '9', '4', '0', '5', '4',
        '4', '9', '2', '1', '9', '4', '8', '1', '5', '9', '7', '9', '9',
        '4', '9', '2', '3', '4', '7', '6', '4', '9', '0', '5', '8', '5',
        '6', '6', '5', '7', '8', '1', '0', '1', '6', '4', '6', '7', '7',
        '1', '7', '1', '8', '2', '0', '1', '9', '9', '5', '5', '1', '5',
        '6', '0', '5', '4', '4', '6', '5', '4', '4', '5', '4', '5', '1',
        '9', '9', '7', '3', '5', '2', '1', '1', '9'

In [60]:
# 결과 확인, 결과 리포트 → 성능 평가
ac_score = metrics.accuracy_score( test['labels'], predict )
ac_score

0.788

In [61]:
cl_report = metrics.classification_report( test['labels'], predict )
print(cl_report)

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        19
          1       0.79      1.00      0.88        34
          2       0.70      0.67      0.68        24
          3       0.86      0.52      0.65        23
          4       0.79      0.79      0.79        33
          5       0.68      0.84      0.75        25
          6       1.00      0.64      0.78        22
          7       0.81      0.76      0.79        29
          8       0.92      0.79      0.85        14
          9       0.65      0.81      0.72        27

avg / total       0.80      0.79      0.79       250



In [62]:
# 풀 세트 구성 후 → 반복작업

decode_mnist_rawData( dataType='train', maxCnt = 60000 )
decode_mnist_rawData( dataType='t10k',  maxCnt = 10000 )

# 알고리즘 만들고
clf = svm.SVC()

train = load_csv()
test = load_csv( dataType = 't10k' )

clf.fit( train['images'], train['labels'] )

predict = clf.predict( test['images'] )

ac_score = metrics.accuracy_score( test['labels'], predict )
print(ac_score)

cl_report = metrics.classification_report( test['labels'], predict )
print(cl_report)

2049 60000
2051 60000 28 28
2049 10000
2051 10000 28 28
0.9443
             precision    recall  f1-score   support

          0       0.96      0.99      0.97       980
          1       0.97      0.99      0.98      1135
          2       0.94      0.93      0.93      1032
          3       0.93      0.94      0.93      1010
          4       0.93      0.96      0.94       982
          5       0.93      0.91      0.92       892
          6       0.95      0.97      0.96       958
          7       0.96      0.93      0.94      1028
          8       0.94      0.92      0.93       974
          9       0.94      0.92      0.93      1009

avg / total       0.94      0.94      0.94     10000

